## Merge reference data with prediction agreement

In [1]:
import os
import pandas as pd
import numpy as np
import requests

In [2]:
dataset_file = "../data/output/dataset.tsv"
prediction_dir = "../data/predictions"  # scp -r urano:/projects/CAID2/predictions .
predictions_references_file = "../data/predictions_references.tsv"

assessment_disorder_file = "../data/disorder.analysis.all.dataset.f1s.metrics.csv"
assessment_binding_file = "../data/binding.analysis.all.dataset.f1s.metrics.csv"

alphafold_dir = "../data/alphafold"

# Output
dataset_merge_file = "../data/output/dataset_merge.tsv"

In [3]:
# It assumes one method participate in only one challenge
df_challenge = pd.read_csv(predictions_references_file, sep="\t")
df_challenge
# df_challenge[df_challenge['Method'] == 'ENSHROUD-all']

,Method,disorder,binding
0,AIUPred,1,0
1,AlphaFold-dis,1,0
2,AlphaFold-dis25,1,0
3,ANCHOR2,0,1
4,APOD,1,0
...,...,...,...
64,SETH-1,1,0
65,SPOT-Disorder,1,0
66,SPOT-Disorder-Single,1,0
67,SPOT-Disorder2,1,0


In [27]:
df_1 = pd.read_csv(assessment_disorder_file, index_col=0).reset_index().rename(columns={'index': 'method'})[['method', 'thr']]
df_1['challenge'] = 'disorder'

df_2 = pd.read_csv(assessment_binding_file, index_col=0).reset_index().rename(columns={'index': 'method'})[['method', 'thr']]
df_2['challenge'] = 'binding'
df_2

df_ = pd.concat([df_1, df_2])

df_thresholds = df_.loc[df_['method'].isin(df_challenge[df_challenge['disorder'] == 1]['Method']) & (df_['challenge'] == 'disorder')]
df_thresholds = pd.concat([df_thresholds, df_.loc[df_['method'].isin(df_challenge[df_challenge['binding'] == 1]['Method']) & (df_['challenge'] == 'binding')]])
df_thresholds['thr'].astype('float32')
df_thresholds

,method,thr,challenge
0,ESpritz-D,0.414,disorder
1,ESpritz-X,0.102,disorder
2,ESpritz-N,0.453,disorder
3,DisEMBL-dis465,0.448,disorder
4,DisEMBL-disHL,0.107,disorder
...,...,...,...
22,OPAL,0.535,binding
23,DISOPRED3-bind,0.000,binding
24,ProBiPred-protein,0.002,binding
25,ProBiPred-nucleic,0.216,binding


In [5]:
df_list = []
for pred_file in os.listdir(prediction_dir):
    pred_name = pred_file[:-5]  
    if pred_name in df_thresholds['method'].values:
#         print(pred_name)
        data = []
        with open("{}/{}".format(prediction_dir, pred_file)) as f:
            for line in f:
                if line[0] == ">":
                    disprot_id = line[1:].strip()
                else:
                    data.append([pred_name, disprot_id] + line[:-1].split("\t"))     
        df_ = pd.DataFrame(data, columns=['method', 'disprot_id', 'pos', 'aa', 'score', 'class'])
#         print(df_)
        if df_['score'].values[0] == "":
            df_['class'] = df_['class'].astype(int)
        else:
            if (df_['score'] == "").any():
                print("Empty score", pred_name)
                df_.loc[df_['score'] == "", 'score'] = np.nan
            df_['score'] = df_['score'].astype('float32')
            df_['class'] = 0
            df_.loc[df_['score'] >= df_thresholds.loc[df_thresholds['method'] == pred_name, 'thr'].iloc[0], 'class'] = 1
        df_.drop(columns=['aa', 'score'], inplace=True)
        df_list.append(df_)
    else:
        print("missing method threshold", pred_name)

df_pred = pd.concat(df_list)    
df_pred

missing method threshold ENSHROUD-nucleic
Empty score DisoBindPred
missing method threshold DeepDISOBind-nucleic
missing method threshold DeepDISOBind-protein
missing method threshold ENSHROUD-protein
missing method threshold AIUPred


,method,disprot_id,pos,class
0,flDPtr,DP02342,1,1
1,flDPtr,DP02342,2,1
2,flDPtr,DP02342,3,1
3,flDPtr,DP02342,4,1
4,flDPtr,DP02342,5,1
...,...,...,...,...
301237,DFLpred,DP03746,1280,1
301238,DFLpred,DP03746,1281,1
301239,DFLpred,DP03746,1282,1
301240,DFLpred,DP03746,1283,1


In [6]:
# Fix bug in SETH-O position column
df_pred['pos'] = df_pred['pos'].astype(int)
df_pred.loc[df_pred['method'] == 'SETH-0', ['pos']] += 1

In [7]:
df_pred = pd.merge(df_pred, df_challenge, left_on=['method'], right_on=['Method'], how='left').drop(columns='Method')
df_pred

,method,disprot_id,pos,class,disorder,binding
0,flDPtr,DP02342,1,1,1,0
1,flDPtr,DP02342,2,1,1,0
2,flDPtr,DP02342,3,1,1,0
3,flDPtr,DP02342,4,1,1,0
4,flDPtr,DP02342,5,1,1,0
...,...,...,...,...,...,...
18581534,DFLpred,DP03746,1280,1,0,1
18581535,DFLpred,DP03746,1281,1,0,1
18581536,DFLpred,DP03746,1282,1,0,1
18581537,DFLpred,DP03746,1283,1,0,1


In [8]:
df_pred_binding = pd.pivot_table(df_pred.loc[df_pred['binding'] == 1], index=['disprot_id', 'pos'], columns=['method'], values=['class'])
df_pred_disorder = pd.pivot_table(df_pred.loc[df_pred['disorder'] == 1], index=['disprot_id', 'pos'], columns=['method'], values=['class'])

In [9]:
df_pred_binding.columns = df_pred_binding.columns.set_levels(['binding'], level=0)
df_pred_disorder.columns = df_pred_disorder.columns.set_levels(['disorder'], level=0)
df_pred = pd.merge(df_pred_disorder, df_pred_binding, on=['disprot_id', 'pos'])
df_pred

disorder                                                   \
method              APOD AUCpred-no-profile AUCpred-profile AlphaFold-dis   
disprot_id pos                                                              
DP02342    1         0.0                1.0             1.0           0.0   
           2         0.0                1.0             1.0           0.0   
           3         0.0                1.0             1.0           0.0   
           4         0.0                1.0             1.0           0.0   
           5         0.0                1.0             1.0           0.0   
...                  ...                ...             ...           ...   
DP03746    1280      0.0                1.0             1.0           NaN   
           1281      0.0                1.0             1.0           NaN   
           1282      0.0                1.0             1.0           NaN   
           1283      0.0                1.0             1.0           NaN   
           1284      0.0                1.0             1.0           NaN   

                                                                          \
method          AlphaFold-dis25 DISOPRED3-diso DeepIDP-2L DisEMBL-dis465   
disprot_id pos                                                             
DP02342    1                1.0            1.0        1.0            1.0   
           2                1.0            0.0        1.0            1.0   
           3                1.0            0.0        1.0            1.0   
           4                1.0            0.0        1.0            1.0   
           5                1.0            0.0        1.0            0.0   
...                         ...            ...        ...            ...   
DP03746    1280             NaN            0.0        1.0            0.0   
           1281             NaN            0.0        1.0            0.0   
           1282             NaN            0.0        1.0            0.0   
           1283             NaN            0.0        1.0            1.0   
           1284             NaN            0.0        1.0            1.0   

                                        ...      binding                \
method          DisEMBL-disHL DisoPred  ... MoRFchibi-mc MoRFchibi-mcl   
disprot_id pos                          ...                              
DP02342    1              1.0      1.0  ...          0.0           1.0   
           2              1.0      1.0  ...          0.0           0.0   
           3              1.0      1.0  ...          0.0           0.0   
           4              1.0      1.0  ...          0.0           0.0   
           5              1.0      1.0  ...          0.0           0.0   
...                       ...      ...  ...          ...           ...   
DP03746    1280           1.0      0.0  ...          1.0           1.0   
           1281           1.0      0.0  ...          1.0           1.0   
           1282           1.0      0.0  ...          1.0           1.0   
           1283           0.0      0.0  ...          1.0           1.0   
           1284           0.0      0.0  ...          1.0           1.0   

                                                                        \
method          MoRFchibi-mcw OPAL ProBiPred-nucleic ProBiPred-protein   
disprot_id pos                                                           
DP02342    1              1.0  1.0               0.0               1.0   
           2              0.0  1.0               0.0               1.0   
           3              0.0  1.0               0.0               1.0   
           4              0.0  1.0               0.0               1.0   
           5              0.0  1.0               1.0               1.0   
...                       ...  ...               ...               ...   
DP03746    1280           1.0  1.0               1.0               1.0   
           1281           1.0  1.0               1.0               1.0   
           1282      

In [10]:
df_dataset = pd.read_csv(dataset_file, sep="\t")
df_dataset

,disprot_id,acc,pos,aa,disorder,linker,transition,nucleic acid binding,binding,protein binding,pdb
0,DP02342,P06837,1,M,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,DP02342,P06837,2,L,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,DP02342,P06837,3,C,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,DP02342,P06837,4,C,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,DP02342,P06837,5,M,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
378005,DP03896,Q13635,1443,G,1.0,NaN,NaN,NaN,NaN,NaN,NaN
378006,DP03896,Q13635,1444,S,1.0,NaN,NaN,NaN,NaN,NaN,NaN
378007,DP03896,Q13635,1445,S,1.0,NaN,NaN,NaN,NaN,NaN,NaN
378008,DP03896,Q13635,1446,S,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# df_dataset.columns.set_levels(['reference'], level=0)
df_dataset.columns = pd.MultiIndex.from_product([['reference'], df_dataset.columns])
df_dataset

reference                                              \
       disprot_id     acc   pos aa disorder linker transition   
0         DP02342  P06837     1  M      1.0    NaN        NaN   
1         DP02342  P06837     2  L      1.0    NaN        NaN   
2         DP02342  P06837     3  C      1.0    NaN        NaN   
3         DP02342  P06837     4  C      1.0    NaN        NaN   
4         DP02342  P06837     5  M      1.0    NaN        NaN   
...           ...     ...   ... ..      ...    ...        ...   
378005    DP03896  Q13635  1443  G      1.0    NaN        NaN   
378006    DP03896  Q13635  1444  S      1.0    NaN        NaN   
378007    DP03896  Q13635  1445  S      1.0    NaN        NaN   
378008    DP03896  Q13635  1446  S      1.0    NaN        NaN   
378009    DP03896  Q13635  1447  N      1.0    NaN        NaN   

                                                         
       nucleic acid binding binding protein binding pdb  
0                       NaN     NaN             NaN NaN  
1                       NaN     NaN             NaN NaN  
2                       NaN     NaN             NaN NaN  
3                       NaN     NaN             NaN NaN  
4                       NaN     NaN             NaN NaN  
...                     ...     ...             ...  ..  
378005                  NaN     NaN             NaN NaN  
378006                  NaN     NaN             NaN NaN  
378007                  NaN     NaN             NaN NaN  
378008                  NaN     NaN             NaN NaN  
378009                  NaN     NaN             NaN NaN  

[378010 rows x 11 columns]

## AlphaFold

Generate AlphaFold predictions, see "alphafold" notebook

In [13]:
df_af = pd.read_csv("{}/af_pred.tsv".format(alphafold_dir), sep="\t")
df_af = df_af.rename(columns={"name": "acc", "disorder": "af-disorder", "disorder-25": "af-rsa", "binding-25-0.581": "af-binding"})
df_af = df_af[["acc", "pos", "aa", "af-disorder", "af-rsa", "af-binding"]]
df_af = df_af.set_index(['acc', 'pos', 'aa'])
df_af.columns = pd.MultiIndex.from_product([['alphafold'], df_af.columns])
df_af

alphafold                  
                  af-disorder af-rsa af-binding
acc        pos aa                              
A0A090N8E9 1   M        0.672  0.792      0.718
           2   G        0.642  0.784      0.731
           3   Q        0.693  0.756      0.710
           4   T        0.646  0.756      0.729
           5   G        0.692  0.769      0.710
...                       ...    ...        ...
S8F2K7     519 E        0.663  0.770      0.722
           520 R        0.685  0.780      0.713
           521 Q        0.706  0.792      0.704
           522 H        0.657  0.767      0.725
           523 A        0.701  0.770      0.706

[279964 rows x 3 columns]

In [14]:
df_dataset = pd.merge(df_dataset, df_af, left_on=[('reference', 'acc'), ('reference', 'pos'), ('reference', 'aa')], right_on=['acc', 'pos', 'aa'], how='left')
df_dataset

reference                                              \
       disprot_id     acc   pos aa disorder linker transition   
0         DP02342  P06837     1  M      1.0    NaN        NaN   
1         DP02342  P06837     2  L      1.0    NaN        NaN   
2         DP02342  P06837     3  C      1.0    NaN        NaN   
3         DP02342  P06837     4  C      1.0    NaN        NaN   
4         DP02342  P06837     5  M      1.0    NaN        NaN   
...           ...     ...   ... ..      ...    ...        ...   
378005    DP03896  Q13635  1443  G      1.0    NaN        NaN   
378006    DP03896  Q13635  1444  S      1.0    NaN        NaN   
378007    DP03896  Q13635  1445  S      1.0    NaN        NaN   
378008    DP03896  Q13635  1446  S      1.0    NaN        NaN   
378009    DP03896  Q13635  1447  N      1.0    NaN        NaN   

                                                          alphafold         \
       nucleic acid binding binding protein binding pdb af-disorder af-rsa   
0                       NaN     NaN             NaN NaN       0.270  0.897   
1                       NaN     NaN             NaN NaN       0.266  0.891   
2                       NaN     NaN             NaN NaN       0.256  0.885   
3                       NaN     NaN             NaN NaN       0.296  0.878   
4                       NaN     NaN             NaN NaN       0.292  0.873   
...                     ...     ...             ...  ..         ...    ...   
378005                  NaN     NaN             NaN NaN       0.618  0.907   
378006                  NaN     NaN             NaN NaN       0.648  0.911   
378007                  NaN     NaN             NaN NaN       0.676  0.910   
378008                  NaN     NaN             NaN NaN       0.636  0.918   
378009                  NaN     NaN             NaN NaN       0.670  0.914   

                   
       af-binding  
0           0.887  
1           0.889  
2           0.893  
3           0.876  
4           0.878  
...           ...  
378005      0.741  
378006      0.729  
378007      0.717  
378008      0.733  
378009      0.719  

[378010 rows x 14 columns]

In [15]:
df_dataset = pd.merge(df_dataset, df_pred, left_on=[('reference', 'disprot_id'), ('reference', 'pos')], right_on=['disprot_id', 'pos'], how='left')
df_dataset

reference                                              \
       disprot_id     acc   pos aa disorder linker transition   
0         DP02342  P06837     1  M      1.0    NaN        NaN   
1         DP02342  P06837     2  L      1.0    NaN        NaN   
2         DP02342  P06837     3  C      1.0    NaN        NaN   
3         DP02342  P06837     4  C      1.0    NaN        NaN   
4         DP02342  P06837     5  M      1.0    NaN        NaN   
...           ...     ...   ... ..      ...    ...        ...   
378005    DP03896  Q13635  1443  G      1.0    NaN        NaN   
378006    DP03896  Q13635  1444  S      1.0    NaN        NaN   
378007    DP03896  Q13635  1445  S      1.0    NaN        NaN   
378008    DP03896  Q13635  1446  S      1.0    NaN        NaN   
378009    DP03896  Q13635  1447  N      1.0    NaN        NaN   

                                                     ...      binding  \
       nucleic acid binding binding protein binding  ... MoRFchibi-mc   
0                       NaN     NaN             NaN  ...          0.0   
1                       NaN     NaN             NaN  ...          0.0   
2                       NaN     NaN             NaN  ...          0.0   
3                       NaN     NaN             NaN  ...          0.0   
4                       NaN     NaN             NaN  ...          0.0   
...                     ...     ...             ...  ...          ...   
378005                  NaN     NaN             NaN  ...          NaN   
378006                  NaN     NaN             NaN  ...          NaN   
378007                  NaN     NaN             NaN  ...          NaN   
378008                  NaN     NaN             NaN  ...          NaN   
378009                  NaN     NaN             NaN  ...          NaN   

                                                                             \
       MoRFchibi-mcl MoRFchibi-mcw OPAL ProBiPred-nucleic ProBiPred-protein   
0                1.0           1.0  1.0               0.0               1.0   
1                0.0           0.0  1.0               0.0               1.0   
2                0.0           0.0  1.0               0.0               1.0   
3                0.0           0.0  1.0               0.0               1.0   
4                0.0           0.0  1.0               1.0               1.0   
...              ...           ...  ...               ...               ...   
378005           NaN           NaN  NaN               NaN               NaN   
378006           NaN           NaN  NaN               NaN               NaN   
378007           NaN           NaN  NaN               NaN               NaN   
378008           NaN           NaN  NaN               NaN               NaN   
378009           NaN           NaN  NaN               NaN               NaN   

                                                        \
       bindEmbed21IDR-idrGeneral bindEmbed21IDR-idrNuc   
0                            1.0                   1.0   
1                            1.0                   1.0   
2                            1.0                   1.0   
3                            1.0                   1.0   
4                            1.0                   1.0   
...                          ...                   ...   
378005                       NaN                   NaN   
378006                       NaN                   NaN   
378007                       NaN                   NaN   
378008                       NaN                   NaN   
378009                       NaN                   NaN   

                                                        
       bindEmbed21IDR-rawGeneral bindEmbed21IDR-rawNuc  
0                            1.0                   1.0  
1                            1.0                   1.0  
2                            1.0                   1.0  
3                            1.0                   1.0  
4                            1.0                   1.0  
...                          .

In [16]:
df_dataset.set_index([('reference', 'disprot_id'), ('reference', 'acc'), ('reference', 'pos'), ('reference', 'aa')], append=True, inplace=True)
df_dataset.index.names = [None, 'disprot_id', 'acc', 'pos', 'aa']
df_dataset

reference                    \
                                  disorder linker transition   
       disprot_id acc    pos  aa                               
0      DP02342    P06837 1    M        1.0    NaN        NaN   
1      DP02342    P06837 2    L        1.0    NaN        NaN   
2      DP02342    P06837 3    C        1.0    NaN        NaN   
3      DP02342    P06837 4    C        1.0    NaN        NaN   
4      DP02342    P06837 5    M        1.0    NaN        NaN   
...                                    ...    ...        ...   
378005 DP03896    Q13635 1443 G        1.0    NaN        NaN   
378006 DP03896    Q13635 1444 S        1.0    NaN        NaN   
378007 DP03896    Q13635 1445 S        1.0    NaN        NaN   
378008 DP03896    Q13635 1446 S        1.0    NaN        NaN   
378009 DP03896    Q13635 1447 N        1.0    NaN        NaN   

                                                                               \
                                 nucleic acid binding binding protein binding   
       disprot_id acc    pos  aa                                                
0      DP02342    P06837 1    M                   NaN     NaN             NaN   
1      DP02342    P06837 2    L                   NaN     NaN             NaN   
2      DP02342    P06837 3    C                   NaN     NaN             NaN   
3      DP02342    P06837 4    C                   NaN     NaN             NaN   
4      DP02342    P06837 5    M                   NaN     NaN             NaN   
...                                               ...     ...             ...   
378005 DP03896    Q13635 1443 G                   NaN     NaN             NaN   
378006 DP03896    Q13635 1444 S                   NaN     NaN             NaN   
378007 DP03896    Q13635 1445 S                   NaN     NaN             NaN   
378008 DP03896    Q13635 1446 S                   NaN     NaN             NaN   
378009 DP03896    Q13635 1447 N                   NaN     NaN             NaN   

                                       alphafold                    ...  \
                                 pdb af-disorder af-rsa af-binding  ...   
       disprot_id acc    pos  aa                                    ...   
0      DP02342    P06837 1    M  NaN       0.270  0.897      0.887  ...   
1      DP02342    P06837 2    L  NaN       0.266  0.891      0.889  ...   
2      DP02342    P06837 3    C  NaN       0.256  0.885      0.893  ...   
3      DP02342    P06837 4    C  NaN       0.296  0.878      0.876  ...   
4      DP02342    P06837 5    M  NaN       0.292  0.873      0.878  ...   
...                               ..         ...    ...        ...  ...   
378005 DP03896    Q13635 1443 G  NaN       0.618  0.907      0.741  ...   
378006 DP03896    Q13635 1444 S  NaN       0.648  0.911      0.729  ...   
378007 DP03896    Q13635 1445 S  NaN       0.676  0.910      0.717  ...   
378008 DP03896    Q13635 1446 S  NaN       0.636  0.918      0.733  ...   
378009 DP03896    Q13635 1447 N  NaN       0.670  0.914      0.719  ...   

                                      binding                              \
                                 MoRFchibi-mc MoRFchibi-mcl MoRFchibi-mcw   
       disprot_id acc    pos  aa                                            
0      DP02342    P06837 1    M           0.0           1.0           1.0   
1      DP02342    P06837 2    L           0.0           0.0           0.0   
2      DP02342    P06837 3    C           0.0           0.0           0.0   
3      DP02342    P06837 4    C           0.0           0.0           0.0   
4      DP02342    P06837 5    M           0.0           0.0           0.0   
...                                       ...           ...           ...   
378005 DP03896    Q13635 1443 G           NaN           NaN           NaN   
378006 DP03896    Q13635 1444 S           NaN           NaN           NaN   
378007 DP03896    Q13635 1445 S           NaN           NaN           NaN   
378008 DP03896    Q13635 1

In [ ]:
df_dataset.to_csv(dataset_merge_file, sep="\t")